In [1]:
!pip install -r requirements.txt

In [2]:
import glob
import os
import json
import langchain
import chromadb

%reload_ext dotenv 
%dotenv 

%run YandexEmbeddings.ipynb

api_key = os.getenv('YAGPT_API_KEY')
folder_id = os.getenv('YA_FOLDER_ID')

### Получили эмбединги от YandexGPT

In [3]:
embeding = YandexEmbeddings(api_key, folder_id)

doc_files = glob.glob('/home/ashneider/mount/simple_rag_yandexgpt/documents/*.txt')
doc_texts = []
for file_name in doc_files:
    file_data = open(file_name, "r").read()
    doc_texts.append(file_data)

embeddings_answer = []
for doc in doc_texts:
    embededing = embeding.embed_documents(doc)
    embeddings_answer.append(embededing)

# print(embeddings_answer) #список строк
# print("__________________________")
# print(embeddings_answer[0]) #строка, внутри которой json
# print("__________________________")
# print(json.loads(embeddings_answer[0])) # dict

doc_embeddings = []
for answer in embeddings_answer:
    doc_embeddings.append(json.loads(answer)["embedding"])

### Положили документы и их эмбединги в БД

In [4]:
client = chromadb.Client()

collection = client.get_or_create_collection(name="InternalDocs")

collection.add(
    embeddings = doc_embeddings,
    documents = [doc_texts[0], doc_texts[1]],
    metadatas = [{"source": "register_account"},{"source": "vocation_request"}],
    ids = ["id1", "id2"]
)

In [24]:
#client.delete_collection("InternalDocs")

### Получили эмбединг вопроса и понему нашли в БД подходящий документ 

In [28]:
from langchain.docstore.document import Document

query = "Как оформить заявку на 18 рабочих дней?"

query_embedding = json.loads(embeding.embed_query(doc))["embedding"]

found_doc = collection.query(
    query_embeddings=query_embedding,
    n_results=1
)

found_doc_lgch =  Document(page_content=found_doc['documents'][0][0], metadata=found_doc['metadatas'][0][0])